In [ ]:
import pandas as pd
import regex as re
from datetime import datetime
import numpy as np

## Method1
Converted Excel data file into CSV and then importing into Python 

In [ ]:
Exit_Notes=pd.read_csv('../data/Exit_Notes.csv',encoding='cp1252')

In [ ]:
Exit_Notes.head()


In [ ]:
Exit_Notes[['POE','Child ID','DOB','Initial IFSP Date','Exit Reason','Exit Date',
            'Days from I-IFSP to Exit','Exit Note']]

## Method 2
Reading the Excel file directly into Python

In [ ]:
Data_File = pd.read_excel('../data/TEIS Exit Data for NSS.xlsx', sheet_name = None) #imports all data

In [ ]:
 Data_File.keys()

In [ ]:
Data_File.values()

## Reading a Sheet from Excel 

In [ ]:
Exit_Notes=Data_File['Exit Notes']
Exit_Notes_Original=Data_File['Exit Notes']  # Copy of original Data Sheet 

In [ ]:
ECO_with_Exit20_21=Data_File['ECO with Exit20-21']
ECO_with_Exit20_21_Original=Data_File['ECO with Exit20-21'] # Copy of original Data Sheet

In [ ]:
Exit_Notes

In [ ]:
Exit_Notes=Exit_Notes[['POE','Child ID','DOB','Initial IFSP Date','Exit Reason','Exit Date',
            'Days from I-IFSP to Exit','Exit Note']]

## Renaming Columns

In [ ]:
Exit_Notes=Exit_Notes.rename(columns={'Initial IFSP Date':'Init_IFSP_Dt','Child ID':'ID','Exit Reason':'Exit_Rsn',
                          'Exit Date':'Exit_Dt','Days from I-IFSP to Exit':'Service_Days'})

In [ ]:
Exit_Notes.loc[Exit_Notes['Exit Note'].isna()]

## Create a new column Actual Service Days

In [ ]:
 Exit_Notes.insert(7,"Actual_Service_Days",0, False)

In [ ]:
Exit_Notes.head()

## Taking a copy of original Df

In [ ]:
Exit_Notes_Copy = Exit_Notes.copy()

In [ ]:
Exit_Notes_Copy.head()

## Handling blank Exit Notes

In [ ]:
for index,row  in Exit_Notes.iterrows():
   if pd.isna(row['Exit Note']) :
        #Exit_Notes.Actual_Service_Days=Exit_Notes.Service_Days
        Exit_Notes.loc[index,'Actual_Service_Days']= Exit_Notes.loc[index,'Service_Days']
               #print(row['Exit Note'],row['Service_Days'],row['Actual_Service_Days'])
    


In [ ]:
Exit_Notes_Copy.groupby('Actual_Service_Days')['ID'].count()

In [ ]:
(Exit_Notes['Actual_Service_Days'] != 0).sum()

In [ ]:
Exit_Notes=Exit_Notes.rename(columns={'Exit Note':'Exit_Note'})
Exit_Notes_Copy=Exit_Notes_Copy.rename(columns={'Exit Note':'Exit_Note'})

In [ ]:
Exit_Notes1=Exit_Notes.loc[Exit_Notes['Actual_Service_Days'] == 0]
Exit_Notes1.head()

## Formatting Jupyter Notebook Cell

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
 display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
pd.reset_option('max_columns')

### Calculating Actual Days of Service

In [ ]:
Exit_Reopen=Exit_Notes1[Exit_Notes1['Exit_Note'].str.contains('Exit Date:|Reopened:|Exit Reason:')==True]

In [ ]:
Exit_Reopen.head()

In [ ]:
Exit=Exit_Reopen['Exit_Note'].str.extractall(pat ='Exit Date: (\d{2}\/\d{2}\/\d{4})').unstack()

In [ ]:
Reopen=Exit_Reopen['Exit_Note'].str.extractall(pat ='Reopened: (\d{2}\/\d{2}\/\d{4})').unstack()

In [ ]:
Exit.head()

In [ ]:
from functools import reduce

Exit_Reopen_Unstack=reduce(lambda x,y: pd.merge(x,y, left_index=True, right_index=True, how='inner'), [Exit_Reopen, Exit, Reopen])

In [ ]:
Exit_Reopen_Unstack.head(1)

In [ ]:
Exit_Reopen_Unstack_subset=Exit_Reopen_Unstack.drop(columns=['POE','Exit_Rsn','Exit_Note'])

In [ ]:
Exit_Reopen_Unstack_subset.head(500)

In [ ]:
mapping={'(0, 0)_x':'ExDT1','(0, 1)_x':'ExDT2','(0, 2)_x':'ExDT3','(0, 3)_x':'ExDT4','(0, 4)_x':'ExDT5','(0, 5)_x':'ExDT6'}
         
Exit_Reopen_Unstack_subset = Exit_Reopen_Unstack_subset.rename(columns=mapping)

In [ ]:
mapping={(0, 0):'ReDT1',(0, 1):'ReDT2',(0, 2):'ReDT3',(0, 3):'ReDT4',(0, 4):'ReDT5',(0, 5):'ReDT6'}
Exit_Reopen_Unstack_subset = Exit_Reopen_Unstack_subset.rename(columns=mapping)

In [ ]:
Exit_Reopen_Unstack_subset.head()


## For Analysis
Exit_Reopen_Unstack_subset.to_csv("../data/Exit_Reopen_Unstack_subset.csv", index = False)

In [ ]:
i=0
for index,row  in Exit_Reopen_Unstack_subset.iterrows():
    #if (row['ID'] == 461388):
        IFSP=row['Init_IFSP_Dt'] 
        EXITDT=row['Exit_Dt']
        SD=row['Service_Days']
        ASD=row['Actual_Service_Days']
        #print("#",SD)
        #print("#",ASD)
        Exit= [row['ExDT6'],row['ExDT5'],row['ExDT4'],row['ExDT3'],row['ExDT2'],row['ExDT1']]
        Reopen= [row['ReDT6'],row['ReDT5'],row['ReDT4'],row['ReDT3'],row['ReDT2'],row['ReDT1']]
     
        E=[]
    
        for item in Exit:
            item=pd.to_datetime(item, format='%m/%d/%Y')
            if (pd.notnull(item)) & (item >= IFSP ) :  # no early dates thanIFSP
                if (len(E)==0) & (item < EXITDT) :
                    E.append(item)
                   # print("*",SD)
                   # print("*",ASD)
                elif (len(E) >= 1) & (item < EXITDT):  # No Duplicates between exit date in exit note and Final Exit date
                    if item > E[-1]: # NO duplicate entries between exit dates in exit notes
                        E.append(item)
                       # print("***",SD)
                       # print("***",ASD)
                    else:
                        E=[]
                       # print("**",SD)
                       # print("**",ASD)
                        break
                else :
                    continue
        
        R=[]
        for item in Reopen:
            item=pd.to_datetime(item, format='%m/%d/%Y')
            if (pd.notnull(item)) & (item >= IFSP ):
                if len(R)==0:
                    R.append(item)
                elif (len(R) >= 1) & (item > R[-1]):
                     R.append(item)
                else:
                    R=[]
                    break
       # print(len(R))
        
        if  len(E) == 0 & len(R) == 0 :
            i=i+1
            #print("*************")
            print(row['ID'],len(E),len(R))
            Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(EXITDT-IFSP).days 
        elif len(E) == 1 & len(R) == 1 :
             Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(E[0]-IFSP).days + (EXITDT-R[0]).days
        elif len(E) == 2 & len(R) == 2 : 
             Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(E[0]-IFSP).days+(E[1]-R[0]).days+(EXITDT-R[1]).days
        elif len(E) == 3 & len(R) == 3 : 
             Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(E[0]-IFSP).days+(E[1]-R[0]).days+(E[2]-R[1]).days+(EXITDT-R[2]).days
        elif len(E) == 4 & len(R) == 4:
            Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(E[0]-IFSP).days +(E[1]-R[0]).days+(E[2]-R[1]).days+(E[3]-R[2]).days+(EXITDT-R[3]).days
        elif  len(E) == 5 & len(R) == 5:
             Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(E[0]-IFSP).days+(E[1]-R[0]).days+(E[2]-R[1]).days+(E[3]-R[2]).days+(E[4]-R[3]).days+(EXITDT-R[4]).days
        elif len(E) == 6 & len(R) == 6: 
            Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(E[0]-IFSP).days+(E[1]-R[0]).days+(E[2]-R[1]).days+(E[3]-R[2]).days+(E[4]-R[3]).days+(E[5]-R[4]).days+(EXITDT-R[5]).days
        else:
            #print("***********")
            #print(row['ID'],len(E),len(R))
            Exit_Reopen_Unstack_subset.loc[index,'Actual_Service_Days']=(EXITDT-IFSP).days
        print(i)

In [ ]:
Exit_Reopen_Unstack_subset.loc[Exit_Reopen_Unstack_subset.ID==461388]

Exit_Reopen_Unstack_subset.drop(columns=[0,192])

## NO of records updated for Actual Service days is 504

In [ ]:
Exit_Reopen_Unstack_subset.loc[Exit_Reopen_Unstack_subset.Actual_Service_Days!=Exit_Reopen_Unstack_subset.Service_Days]

In [ ]:
Exit_Reopen_Subset=Exit_Reopen_Unstack_subset[['ID', 'Actual_Service_Days']]

In [ ]:
df = Exit_Notes.merge(Exit_Reopen_Subset, how='left', on='ID')
print(df)


In [ ]:
df['Actual_Service_Days'] = df['Actual_Service_Days_y'].fillna(df['Actual_Service_Days_x'])


In [ ]:
print(df)

In [ ]:
df.loc[df.Actual_Service_Days!=0]

In [ ]:
df.loc[df.ID==477472]

In [ ]:
df['Actual_Service_Days'].unique()

## convert float into int

In [ ]:
df['Actual_Service_Days'] = df['Actual_Service_Days'].astype(int)

In [ ]:
df['Actual_Service_Days'].unique()

In [ ]:
df=df.drop(['Actual_Service_Days_x', 'Actual_Service_Days_y'], axis=1)

## For Analysis

In [ ]:
df_analysis=df.loc[df.Actual_Service_Days==0]

df_analysis.to_csv("../data/df_analysis.csv", index = False)

In [ ]:
print(df_analysis)

In [ ]:
df_analysis['Actual_Service_Days'] = df_analysis.apply(lambda x: x['Service_Days'] if x['Actual_Service_Days'] == 0 else x['Actual_Service_Days'], axis=1)

In [ ]:
df.loc[df.Actual_Service_Days==0]

## Actual Service days= service days for children with  exit notes having none of exit or reopened dates

In [ ]:
df['Actual_Service_Days'] = df.apply(lambda x: x['Service_Days'] if x['Actual_Service_Days'] == 0 else x['Actual_Service_Days'], axis=1)

In [ ]:
Exit_Notes.loc[Exit_Notes.Service_Days<182]

In [ ]:
df.loc[df.Actual_Service_Days<182]

In [ ]:
Exit_Notes_Actual=Exit_Notes_Original.merge(df[['ID','Actual_Service_Days']],how='inner',left_on='Child ID',right_on='ID')

In [ ]:
 Exit_Notes_Actual=Exit_Notes_Actual.drop(columns='ID')


In [ ]:
Exit_Notes_Actual.loc[Exit_Notes_Actual.Actual_Service_Days<182].shape

Exit_Notes_Actual.to_csv('../data/Exit_Notes_Actual.csv')

In [ ]:
Exit_Notes_Original.loc[Exit_Notes_Original['Days from I-IFSP to Exit'] <182].shape

In [ ]:
Exit_Notes_Actual.loc[Exit_Notes_Actual.Actual_Service_Days != Exit_Notes_Original['Days from I-IFSP to Exit'] ].shape

In [ ]:
ECO_with_Exit20_21_Actual=ECO_with_Exit20_21_Original.merge(Exit_Notes_Actual[['Child ID','Actual_Service_Days']],how='left',left_on='CHILD_ID',right_on='Child ID')
#ECO_with_Exit20_21_Original.head()

In [ ]:
ECO_with_Exit20_21_Actual.head()

In [ ]:
ECO_with_Exit20_21_Actual=ECO_with_Exit20_21_Actual.drop(columns='Child ID')

In [ ]:
ECO_with_Exit20_21_Actual.tail(10)

In [ ]:
ECO_with_Exit20_21_Actual.drop(ECO_with_Exit20_21_Actual.tail(7).index,
        inplace = True)

In [ ]:
ECO_with_Exit20_21_Actual.tail(10)

In [ ]:
ECO_with_Exit20_21_Actual['Days btw Initial and Exit'] = ECO_with_Exit20_21_Actual['Days btw Initial and Exit'].astype(int)
ECO_with_Exit20_21_Actual['Actual_Service_Days']=ECO_with_Exit20_21_Actual['Actual_Service_Days'].fillna(ECO_with_Exit20_21_Actual['Days btw Initial and Exit'])
ECO_with_Exit20_21_Actual['Actual_Service_Days'] = ECO_with_Exit20_21_Actual['Actual_Service_Days'].astype(int)

ECO_with_Exit20_21.to_csv('../data/Eco_Analysis.csv',index=False)

In [ ]:
ECO_with_Exit20_21_Actual.loc[ECO_with_Exit20_21_Actual['Actual_Service_Days'] < 182 ].shape

In [ ]:
ECO_with_Exit20_21_Actual.loc[ECO_with_Exit20_21_Actual['Actual_Service_Days'] >= 182 ].shape

ECO_with_Exit20_21_Actual.to_csv('../data/ECO_with_Exit20_21_Actual_Analysis.csv',index=False)